# Summarize large text

<p style="font-size: 20px; font-weight: bold; font-style: italic;">...via LLM-graph</p>

Anton Antonov   
August, September 2025

----

## Introduction

This notebook illustrates how to specify a Large Language Model (LLM) graph for deriving comprehensive summaries of large texts. 
The LLM graph is based on different LLM- and non-LLM functions .
The Raku package ["LLM::Graph"](https://raku.land/zef:antononcube/LLM::Graph) is used, [AAp1].

Using the LLM graph is an alternative to the Literate programming based solutions shown in [AA1, AAn1].

----

## Setup

In [1]:
use LLM::Graph;
use Data::Importers;

use LLM::Tooling;
use WWW::Ollama;

In [2]:
sink my $conf5-mini = llm-configuration('ChatGPT', model => 'gpt-5-mini');
sink my $conf5 = llm-configuration('ChatGPT', model => 'gpt-5');
sink my $conf41-mini = llm-configuration('ChatGPT', model => 'gpt-4.1-mini', temperature => 0.55, max-tokens => 4096);
sink my $conf41 = llm-configuration('ChatGPT', model => 'gpt-4.1', temperature => 0.45, max-tokens => 8192);
sink my $conf4o-mini = llm-configuration('ChatGPT', model => 'gpt-4o-mini', temperature => 0.45, max-tokens => 8192);
sink my $conf4o = llm-configuration('ChatGPT', model => 'gpt-4o', temperature => 0.45, max-tokens => 8192);

In [3]:
sink my $conf-ollama = LLM::Functions::Configuration.new(
        name => 'ollama',
        api-key => Whatever,
        api-user-id => 'user:' ~ ((10 ** 11 + 1) .. 10 ** 12).pick,
        module => 'WWW::Ollama',
        base-url => ollama-base-url,
        model => 'gemma3n',
        function => &ollama-chat-completion,
        embedding-model => 'nomic-embed-text',
        embedding-function => &ollama-embedding,
        temperature => 0.4,
        max-tokens => 8192,
        total-probability-cutoff => 0,
        prompts => Empty,
        prompt-delimiter => ' ',
        examples => Empty,
        stop-tokens => Empty,
        argument-renames => %(),
        format => 'values');

In [4]:
sink my $conf = $conf41-mini;

----

## LLM graph

Specify the LLM graph nodes:

In [5]:
sink my %rules =
TypeOfInput => sub ($_) {
        "Determine the input type of\n\n$_.\n\nThe result should be one of: 'Text', 'URL', 'FilePath', or 'Other'."  ~ 
        llm-prompt('NothingElse')('single string')
    },

IngestText =>  { eval-function => sub ($TypeOfInput, $_) { $TypeOfInput ~~ / URL | FilePath/ ?? data-import($_) !! $_} },

Title => { 
    eval-function => sub ($IngestText, $with-title = Whatever) { $with-title ~~ Str:D ?? $with-title !! llm-synthesize([llm-prompt("TitleSuggest")($IngestText, 'article'), "Short title with less that 6 words"], llm-evaluator => $conf) },
},

Summary => sub ($IngestText) { llm-prompt("Summarize")() ~ "\n\n$IngestText" },

TopicsTable => sub ($IngestText) { llm-prompt("ThemeTableJSON")($IngestText, 'article', 20) },

ThinkingHats => sub ($IngestText) { llm-prompt("ThinkingHatsFeedback")($IngestText, <yellow grey>, format => 'HTML') },

MindMap => sub ($IngestText) { llm-prompt('MermaidDiagram')($IngestText) },

MostProvocative => sub ($IngestText) { 
        "Give a JSON table of the most important or provocative statements in the follow text:\n\n" ~ 
        $IngestText ~
        "\n\n" ~
        llm-prompt('NothingElse')('JSON')
    },

Report => { eval-function => 
    sub ($Title, $Summary, $TopicsTable, $MindMap, $ThinkingHats, $MostProvocative) { 
        [
            "# $Title",
            '### *LLM summary report*',
            '## Summary',
            $Summary,
            '## Topics',
            to-html(
                from-json($TopicsTable.subst(/ ^ '```json' | '```' $/):g),
                field-names => <theme content>,
                align => 'left'),
            "## Mind map",
            $MindMap,
            '## Thinking hats',
            $ThinkingHats.subst(/ ^ '```html' | '```' $/):g,
            '## Most important or provocative statements',
            to-html(
                from-json($MostProvocative.subst(/ ^ '```json' | '```' $/):g),
                align => 'left'),
        ].join("\n\n")
    } 
},

ExportAndOpen => {
    eval-function => sub ($Report) {
       spurt('./Report.md', $Report);
       shell "open ./Report.md" 
    },
    test-function => -> $export-and-open = True { $export-and-open ~~ Bool:D && $export-and-open || $export-and-open.Str.lc ∈ <true yes open> }
}
;

Make the graph:

In [6]:
my $gCombinedSummary = llm-graph(%rules, llm-evaluator => $conf, :!async, :progress)

LLM::Graph(size => 10, nodes => ExportAndOpen, IngestText, MindMap, MostProvocative, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

In [7]:
.say for $gCombinedSummary.nodes

Title => {eval-function => sub { }}
TypeOfInput => sub { }
MindMap => sub { }
TopicsTable => sub { }
MostProvocative => sub { }
Summary => sub { }
Report => {eval-function => sub { }}
IngestText => {eval-function => sub { }}
ThinkingHats => sub { }
ExportAndOpen => {eval-function => sub { }, test-function => -> $export-and-open = Bool::True { #`(Block|3313150621904) ... }}


----

## Full computation

URL and text statistics:

In [8]:
my $url = 'https://raw.githubusercontent.com/antononcube/RakuForPrediction-blog/refs/heads/main/Data/Graph-neat-examples-in-Raku-Set-2-YouTube.txt';
my $txtFocus = data-import($url);

text-stats($txtFocus)

(chars => 5957 words => 1132 lines => 157)

Computation:

In [9]:
$gCombinedSummary.eval({'$_' => $url, with-title => '«Graph» neat examples, set 3'})

LLM::Graph(size => 10, nodes => ExportAndOpen, IngestText, MindMap, MostProvocative, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

**Remark:** Instead of deriving the title using an LLM, the title is specified as an argument.

Show the corresponding graph:

In [10]:
#% html
$gCombinedSummary.dot(node-width => 1.2, theme => 'ortho'):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.1 (20251213.1925)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- $_ -->
 
 $_ 
 
 $_ 
 
<!-- IngestText -->
 
 IngestText 
 
 IngestText 
 
<!-- $_->IngestText -->
 
 $_->IngestText 
 
 
 
<!-- TypeOfInput -->
 
 TypeOfInput 
 
 
 TypeOfInput 
 
<!-- $_->TypeOfInput -->
 
 $_->TypeOfInput 
 
 
 
<!-- ExportAndOpen -->
 
 ExportAndOpen 
 
 ExportAndOpen 
 
<!-- MindMap -->
 
 MindMap 
 
 
 MindMap 
 
<!-- IngestText->MindMap -->
 
 IngestText->MindMap 
 
 
 
<!-- MostProvocative -->
 
 MostProvocative 
 
 
 MostProvocative 
 
<!-- IngestText->MostProvocative -->
 
 IngestText->MostProvocative 
 
 
 
<!-- Summary -->
 
 Summary 
 
 
 Summary 
 
<!-- IngestText->Summary -->
 
 IngestText->Summary 
 
 
 
<!-- ThinkingHats -->
 
 ThinkingHats 
 
 
 ThinkingHats 
 
<!-- IngestText->ThinkingHats -->
 
 IngestText->ThinkingHats 
 
 
 
<!-- Title -->
 
 Title 
 
 Title 
 
<!-- IngestText->Title -->
 
 IngestText->Title 
 
 
 
<!-- TopicsTable -->
 
 TopicsTable 
 
 
 TopicsTable 
 
<!-- IngestText->TopicsTable -->
 
 IngestText->TopicsTable 
 
 
 
<!-- Report -->
 
 Report 
 
 Report 
 
<!-- MindMap->Report -->
 
 MindMap->Report 
 
 
 
<!-- MostProvocative->Report -->
 
 MostProvocative->Report 
 
 
 
<!-- Report->ExportAndOpen -->
 
 Report->ExportAndOpen 
 
 
 
<!-- Summary->Report -->
 
 Summary->Report 
 
 
 
<!-- ThinkingHats->Report -->
 
 ThinkingHats->Report 
 
 
 
<!-- Title->Report -->
 
 Title->Report 
 
 
 
<!-- TopicsTable->Report -->
 
 TopicsTable->Report 
 
 
 
<!-- TypeOfInput->IngestText -->
 
 TypeOfInput->IngestText 
 
 
 
<!-- with-title -->
 
 with-title 
 
 with-title 
 
<!-- with-title->Title -->
 
 with-title->Title

Final result:

In [11]:
#% markdown
$gCombinedSummary.nodes<Report><result>.subst(/'```html' | '```' $/):g

# «Graph» neat examples, set 3

### *LLM summary report*

## Summary

The presentation titled "Graph Neat Examples in Raku" by Anton Antonov demonstrates creating and visualizing various types of graphs using Raku, including undirected triangular grid graphs, bipartite grid graphs, directed graphs, and collections of star graphs. It highlights the use of adjacency and incidence matrices, bipartite coloring, connected components, and visualization techniques with JavaScript D3 and the DOT language from Graphviz. The talk also emphasizes the power and simplicity of the Raku graph package and its integration with visualization tools for producing compelling graphical outputs.

## Topics

<table border="1"><thead><tr><th>theme</th><th>content</th></tr></thead><tbody><tr><td align=left>Introduction and first example</td><td align=left>Presentation introduction, triangular grid graph, vertex degree coloring, adjacency and incidence matrices explained.</td></tr><tr><td align=left>Explanation of NE examples and tools</td><td align=left>Definition of NE examples, use of Raku graph package, JavaScript D3 visualization, and Graphviz dot method.</td></tr><tr><td align=left>Second example: Bipartite grid graphs</td><td align=left>Grid graphs are bipartite, vertex coloring shows sets, verifying bipartiteness and deriving partition coloring.</td></tr><tr><td align=left>Code walkthrough for bipartite graphs</td><td align=left>Generating grid graph, verifying bipartite, using classify function and vertex coordinates for visualization.</td></tr><tr><td align=left>Third example: Directed grid graphs and connected components</td><td align=left>Randomly directed grid graph, connected components coloring, explanation of reachability in directed graphs.</td></tr><tr><td align=left>Visualization methods: JavaScript and DOT</td><td align=left>JavaScript code generation and rendering in Jupyter, SVG visualization with DOT, node shape variations.</td></tr><tr><td align=left>DOT language usage and terminology</td><td align=left>Using dot language for graph styling, difference between vertex and node terminology, shape customization.</td></tr><tr><td align=left>Last example: Collage of star graphs</td><td align=left>Creating 40 star graphs with 8-16 rays, coloring subsets, possibly generated by large language models.</td></tr><tr><td align=left>DOT visualization advantages</td><td align=left>DOT produces more regular plots, style specified via dot language preamble, easier manipulation and styling.</td></tr><tr><td align=left>Conclusion</td><td align=left>Summary and thanks for attention.</td></tr></tbody></table>

## Mind map

```mermaid
mindmap
  root("Presentation Graph Neat")
    introduction("Introduction")
      title("Title Graph Neat")
      author("Anton Antonov")
      date("November 13th 20124")
    example1("Example 1 Undirected Triangular Grid Graph")
      description("Vertices colored by degree")
      adjacencyMatrix("Adjacency Matrix vertex to vertex symmetric")
      incidenceMatrix("Incidence Matrix vertex to edge two non zero elements per column")
    neExamples("NE Examples")
      conciseCode("Concise straightforward code")
      showcaseRaku("Showcase Raku programming")
      showcaseModules("Showcase Raku modules")
      computationalPerspective("Computationally possible perspectives")
    tools("Tools Used")
      graphPackage("Raku graph package")
      visualization("Visualization with JavaScript D3")
      dotMethod("Dot method using Graph Vis dot language")
      svgVisualization("SVG images visualization")
    example2("Example 2 Bipartite Grid Graph")
      bipartiteDefinition("Two sets of vertices no edges inside sets")
      coloring("Bipartite coloring derived")
      code("Generating grid graph and verifying bipartite")
      vertexCoordinates("Using vertex coordinates in graph package")
    example3("Example 3 Directed Random Grid Graph")
      undirectedToDirected("Undirected to directed random")
      connectedComponents("Connected components colored")
      reachability("Vertices reachability explanation")
      visualizationDot("Visualization with dot")
      javascriptVisualization("JavaScript visualization in Jupiter notebook")
    dotLanguage("Dot Language Usage")
      shapeRandom("Random node shapes hexagonal triangular")
      terminology("Vertex and node terminology")
      stylePreamble("Preamble to specify style")
      dotDsl("Dot language DSL powerful and straightforward")
    example4("Example 4 Collage of Star Graphs")
      starGraphs("40 star graphs with 8 to 16 rays")
      coloring("Colored subsets")
      codeGeneration("Code generated by large language models")
      manipulation("Fun manipulating star graphs")
      dotComparison("Dot visualization more regular")
      dotArguments("Style specified in dot language not individual arguments")
    conclusion("Conclusion")
      thanks("Thank you")
```

## Thinking hats

<table>
  <tr>
    <th>Hat Name</th>
    <th>Perspective</th>
    <th>Feedback</th>
    <th>New Ideas</th>
  </tr>
  <tr>
    <td>Yellow Hat</td>
    <td>Benefits and creativity</td>
    <td>This presentation highlights the power of Raku's graph package combined with JavaScript D3 and DOT language for versatile and visually compelling graph visualizations. The use of bipartite coloring and connected components visualization offers clear, intuitive insights into graph structures. The integration with Jupyter notebooks enhances accessibility and interactivity, making it a great tool for teaching, research, and exploratory data analysis. The modular approach allows for creative extensions, such as random directed graphs and star graph collages, showcasing versatility and engagement.</td>
    <td>Explore interactive web apps for live graph manipulation, integrate machine learning to detect graph properties automatically, develop tutorials targeting different expertise levels to broaden adoption.</td>
  </tr>
  <tr>
    <td>Gray Hat</td>
    <td>Cynicism and skepticism</td>
    <td>While the presentation sounds impressive, there's an underlying risk of overhyping the ease and power of these visualizations. The reliance on multiple technologies (Raku, JavaScript D3, DOT) may mask complexity and steep learning curves that aren't fully acknowledged. Also, the claim that code was generated by large language models but not shown might hint at a lack of transparency or overreliance on AI-generated content without thorough vetting. The naming inconsistencies between "vertex" and "node" suggest a potential messiness in terminology that could confuse users.</td>
    <td>Clarify limitations and challenges upfront, provide full reproducible code examples including AI-generated parts, standardize terminology to avoid confusion.</td>
  </tr>
</table>

## Most important or provocative statements

<table border="1"><thead><tr><th>statement</th></tr></thead><tbody><tr><td align=left>The adjacency Matrix is vertex to vertex it&#39;s symmetric this is an undirected graph.</td></tr><tr><td align=left>The incidence Matrix shows the vertex to Edge relationship so each column corresponds to an edge and we can see each column has exactly two elements non zero elements.</td></tr><tr><td align=left>NE examples are concise or straightforward code that produce compelling visual or textual outputs.</td></tr><tr><td align=left>They showcase Raku programming and functionalities of different Raku modules and give interesting perspectives on what is computationally possible.</td></tr><tr><td align=left>Grid graphs are bipartite meaning two sets of nodes where vertices in the same set do not connect but connect to the other set.</td></tr><tr><td align=left>Finding connected components in a directed graph is a much more complicated problem than in an undirected graph.</td></tr><tr><td align=left>Connected components are visualized by coloring vertices differently to show reachability within the components.</td></tr><tr><td align=left>Graph Vis&#39;s dot language can produce SVG images and is installable on almost any system.</td></tr><tr><td align=left>JavaScript visualization using D3 can be rendered straightforwardly in Jupyter notebooks using web-based technology.</td></tr><tr><td align=left>The dot method allows specifying vertex or node shapes and uses the dot language DSL for powerful and straightforward graph styling.</td></tr><tr><td align=left>A collage of 40 star graphs with 8 to 16 rays each was created and colored, possibly generated using large language models.</td></tr><tr><td align=left>Using the dot language DSL for graph styling is not difficult to learn and allows specifying styles in a preamble reused for plots.</td></tr></tbody></table>

---

## Partial evaluation

Drop the results in `LLM::Graph` computed above:

In [12]:
$gCombinedSummary.clear

LLM::Graph(size => 10, nodes => ExportAndOpen, IngestText, MindMap, MostProvocative, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

Here the are normalized nodes without results:

In [13]:
.say for |$gCombinedSummary.nodes

Title => {eval-function => sub { }, spec-type => (Callable)}
TypeOfInput => {eval-function => sub { }, spec-type => (Routine), wrapper => Routine::WrapHandle.new}
MindMap => {eval-function => sub { }, spec-type => (Routine), wrapper => Routine::WrapHandle.new}
TopicsTable => {eval-function => sub { }, spec-type => (Routine), wrapper => Routine::WrapHandle.new}
MostProvocative => {eval-function => sub { }, spec-type => (Routine), wrapper => Routine::WrapHandle.new}
Summary => {eval-function => sub { }, spec-type => (Routine), wrapper => Routine::WrapHandle.new}
Report => {eval-function => sub { }, spec-type => (Callable)}
IngestText => {eval-function => sub { }, spec-type => (Callable)}
ThinkingHats => {eval-function => sub { }, spec-type => (Routine), wrapper => Routine::WrapHandle.new}
ExportAndOpen => {eval-function => sub { }, spec-type => (Callable), test-function => -> $export-and-open = Bool::True { #`(Block|3313150621904) ... }}


Here all results are pre-assigned as arguments:

In [14]:
$gCombinedSummary.eval(
    '$_' => $url, 
    Title => '«Graph» neat examples, set 3', 
    :export-and-open,
    TypeOfInput => 'Other',
    Summary => 'In brief',
    IngestText => 'Ingest text',
    TopicsTable => '["TopicsTable"]',
    ThinkingHats => 'Thinking hats',
    MindMap => 'Mind-map graph'
)

LLM::Graph(size => 10, nodes => ExportAndOpen, IngestText, MindMap, MostProvocative, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

-----

## References

### Blog posts

[AA1] Anton Antonov,
["Parameterized Literate Programming"](https://rakuforprediction.wordpress.com/2025/06/21/parameterized-literate-programming/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

### Notebooks

[AAn1] Anton Antonov,
["LLM comprehensive summary template for large texts"](https://community.wolfram.com/groups/-/m/t/3448842),
(2025),
[Wolfram Community](https://community.wolfram.com).

### Packages

[AAp1] Anton Antonov, 
[LLM::Graph, Raku package](https://github.com/antononcube/Raku-LLM-Graph),
(2025),
[GitHub/antononcube](https://github.com/antononcube).